Summer Gregurek and Siyuan Xuan's Project: The Last Astronaut, a Pygame

The following code is a platformer game similar to the original Marios Brothers game and loosely based off of a game that was written in C for EE 319K. The Last Astronaut started as a project to learn how to translate C into Python using existing libraries and tools, and evolved into a unique Python game that runs in simulation on the computer. The entirety of its code is included below, including both classes that make up the game play functionality as it sits, and classes that we programmed to learn more about the Pygame library but did not make it into the final version of the game. The game will not run in the Jupyter notebook, as it needs dependency image files for the sprites, so the whole game folder that will run will be turned in on Canvas with this notebook. We will also include a link to our GitHub, so that the evolution of this game can be seen as it was being programmed. The original C source code is also staged on the GitHub under the folder "Lab 10". Additionally, we will provide a link to YouTube video that shows demonstration of the original puzzle platformer in C so that the evolution of the game from a puzzle with stationary obstacles to a Mario-esque platformer with a moving alien enemy can be seen. Finally, we will provide links to the forum with the walkthrough that gave us ideas for much of the structure of the game itself, as it was noted on Canvas to "Please include your resources (It is ok if you found a solution for your problem on the web, I consider this to be clever.)" Although we used this walkthrough as a starting point for our game, we changed a lot about how the platforms themselves work, and we used original graphics created by us. We also included a starting and ending screen to our game, and changed spawn mechanics, character speed, and how levels work within the game.

The game is played by using the arrow keys to move the astronaut left and right and using the spacebar to jump. The astronaut can collide with the platforms in the game, the coins, and the enemy space alien. The astronaut must collect ten coins to win the game before he is killed five times by the alien. The alien moves randomly at a speed that varies and if it touches the player, both the player and alien will respawn at their respective starting points and the player will lose a life. The loss of life will be reflected by the status icons in the top left corner of the screen. If the alien touches a coin, the coin will move positions randomly. If the player touches a coin, the player collects it and it will be shown in inventory in the top right corner. Whenever the player collects all the coins and wins, a winning screen will show and prompt the player to restart or quit by clicking a button with the mouse. Likewise, whenever the player dies five times to the space alien, a losing screen will show and also prompt the player to restart or quit.

Our code is included below:

In [ ]:
import pygame, sys, random
pygame.init()
X = 800
Y = 640

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 155, 0)
GREY = (128, 128, 128)
screen = pygame.display.set_mode((X, Y))
pygame.display.set_caption("Pygame by Siyuan and Summer")
clock = pygame.time.Clock()
pygame.time.set_timer(pygame.USEREVENT + 1, 10000)
class Platform():
    rects = []
    def __init__(self, posx, posy):
        self.surf = pygame.image.load('WallPy.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass
    
class Spike():
    def __init__(self, posx, posy, direction):#0:up 1:right 2:down 3:left
        if direction==0:
            self.surf = pygame.image.load('Spike0.bmp').convert()
        elif direction==1:
            self.surf = pygame.image.load('Spike90.bmp').convert()
        elif direction==2:
            self.surf = pygame.image.load('Spike180.bmp').convert()
        else:
            self.surf = pygame.image.load('Spike270.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass
    
class Exit():
    rects = []
    def __init__(self, posx, posy):
        self.surf = pygame.surface.Surface((40, 40))
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        self.surf.fill(GREY)
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass
    
class Player():
    def __init__(self):
        self.jump = False
        self.left = False
        self.right = False
        self.surf = pygame.image.load('AstroRLarge.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(60,600))
        self.y_speed = 0
    def event(self):
        if self.jump:
            self.y_speed = -22
            self.jump = False
        if self.left and self.rect.left > 0:
            self.rect.centerx -= 5
        if self.right and self.rect.right < X:
            self.rect.centerx += 5
        self.rect.bottom += self.y_speed
        if self.on_ground():
            if self.y_speed >= 0:
                self.rect.bottom = Platform.rects[self.rect.collidelist(Platform.rects)].top + 1
                self.y_speed = 0
            else:
                self.rect.top = Platform.rects[self.rect.collidelist(Platform.rects)].bottom
                self.y_speed = 2
        else:
            self.y_speed += game.acc
    def on_ground(self):
        collision = self.rect.collidelist(Platform.rects)
        if collision > -1:
            return True
        else:
            return False
    def draw(self):
        screen.blit(self.surf, self.rect)
        
class Enemy():
    def __init__(self):
        self.surf = pygame.image.load('Alien0.bmp').convert()
        self.rect = self.surf.get_rect(midtop=(700,500))
        self.x_speed = random.randint(3, 7)
        self.y_speed = 0
    def event(self):
        self.rect.centerx += self.x_speed
        if self.rect.left <= 0 or self.rect.right >= X:
            self.x_speed *= -1
        if self.on_ground():
            self.rect.bottom = Platform.rects[self.rect.collidelist(Platform.rects)].top + 1
            self.y_speed = 0
        else:
            self.y_speed += game.acc
        self.rect.bottom += self.y_speed
        self.hit()
        if game.timer:
            game.timer = False
            self.rect.midtop = (X//2, 0)
            self.x_speed = random.randint(3, 7) * ((self.x_speed > 0) - (self.x_speed < 0))

    def on_ground(self):
        collision = self.rect.collidelist(Platform.rects)
        if collision > -1:
            return True
        else:
            return False
    def hit(self):
        if (game.player.rect.colliderect(self.rect) and
            game.player.rect.midbottom != (60, 600 )):
            game.lives -= 1
            game.player.rect.midbottom = (60, 600)
            self.rect.midbottom = (420, 140)
    def draw(self):
        screen.blit(self.surf, self.rect)

class Coin():
    def __init__(self):
        self.positions = [(460,440), (280,480), (60, 600), (700, 600), (540, 320), (60,120), (140, 120)]
        self.surf = pygame.image.load('coin0.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=random.choice(self.positions))
        self.count = 0
    def event(self):
        if game.player.rect.colliderect(self.rect):
            self.rect.midbottom = random.choice(self.positions)
            game.coin_count += 1
        elif game.enemy.rect.colliderect(self.rect):
            self.rect.midbottom = random.choice(self.positions)
    def draw(self):
        screen.blit(self.surf, self.rect)
        
class Game():
    def __init__(self):
        self.heart_surf = pygame.image.load('SpriteAstroR0.bmp').convert()
        self.coin_surf = pygame.transform.scale(pygame.image.load('coin0.bmp').convert(), (20, 20))
        self.acc = 2
        self.timer = False
        self.level = 0
        self.state = self.startpage
    def init(self):
        self.lives = 5
        self.coin_count = 0
        self.sprites = [self]
        self.state = self.loop
        self.player = Player()
        self.sprites.append(self.player)
        self.enemy = Enemy()
        self.sprites.append(self.enemy)
        self.coin = Coin()
        self.sprites.append(self.coin)
        
        for i in range(20):
            self.sprites.append(Platform(20+i*40,640))
        for i in range(13):
            self.sprites.append(Platform(20+i*40, 160))
        for i in range(4):
            self.sprites.append(Platform(500+i*40,360))
            self.sprites.append(Platform(580+i*40, 240))
        for i in range(6):
            self.sprites.append(Platform(120+i*40,520))
        for i in  range (7):
            self.sprites.append(Platform(420+i*40,480))
            self.sprites.append(Platform(20+i*40,360))
        
    
    def event(self):
        " a game state function "
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and self.player.on_ground():
                    self.player.jump = True
            elif event.type == pygame.USEREVENT + 1:
                self.timer = True
        self.player.left = False
        self.player.right = False
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.player.left = True
        if keys[pygame.K_RIGHT]:
            self.player.right = True
    def draw(self):
        for i in range(self.lives):
            screen.blit(self.heart_surf, [i*20 + 20, 20])
        for i in range(self.coin_count):
            screen.blit(self.coin_surf, [750 - i*20, 20])

    def loop(self):
        " a game state function "
        clock.tick(30)
        screen.fill(GREY)
        for s in self.sprites:
            s.event()
        for s in self.sprites:
            s.draw()
        pygame.display.flip()
        if self.lives == 0:
            self.state = self.play_again_lose
        if self.coin_count == 10:#######################level up condition
            self.state = self.play_again_win
    def play_again_win(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Win!", 1, GREEN)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click
    def play_again_lose(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Lose...", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click
    def mouse_click(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.state = self.end
                    return
                elif event.type == pygame.MOUSEBUTTONDOWN and pygame.mouse.get_pressed() == (1, 0, 0):
                    pos = pygame.mouse.get_pos()
                    for btn, action in self.buttons:
                        if btn.collidepoint(pos):
                            self.state = action
                            return
            pygame.time.wait(20)
    def end(self):
        pygame.quit()
        sys.exit()
    def startpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("The Last Astronaut", 1, BLUE)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Now", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(play, self.init)]
        pygame.display.flip()
        self.state = self.mouse_click
    def endpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("Thank you for playing", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        stop = button("Exit", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(stop, self.end)]
        pygame.display.flip()
        self.state = self.mouse_click
def button(txt, color, pos):
    button_font = pygame.font.SysFont("Segoe Print", 16)
    btn_surf = pygame.Surface((100, 40))
    btn_rect = btn_surf.get_rect(topleft=(pos))
    btn_surf.fill(GREY)
    pygame.draw.rect(btn_surf, BLACK, (0, 0, 100, 40), 1)
    txt_surf = button_font.render(txt, 1, color)
    txt_rect = txt_surf.get_rect(center=(50,20))
    btn_surf.blit(txt_surf, txt_rect)
    screen.blit(btn_surf, btn_rect)
    return btn_rect

game = Game()

while True:
    game.state()

We will explain parts of this code starting from the top:

In [ ]:
import pygame, sys, random
pygame.init()
X = 800
Y = 640

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 155, 0)
GREY = (128, 128, 128)
screen = pygame.display.set_mode((X, Y))
pygame.display.set_caption("Pygame by Siyuan and Summer")
clock = pygame.time.Clock()
pygame.time.set_timer(pygame.USEREVENT + 1, 10000)

The above code is some initializations that import libraries, including Pygame, set the size of the screen, which was calculated to be five times the scale of the original LCD, some color initializations, screen display in simulation with our names showing in the window's caption, and finally an in-game timer for events.

In [ ]:
class Platform():
    rects = []
    def __init__(self, posx, posy):
        self.surf = pygame.image.load('WallPy.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass

The above code is our Platform class that puts our wall graphic into blocks with which our sprites (coins, astronaut, and alien) can stand on top of. 

In [ ]:
class Spike():
    def __init__(self, posx, posy, direction):#0:up 1:right 2:down 3:left
        if direction==0:
            self.surf = pygame.image.load('Spike0.bmp').convert()
        elif direction==1:
            self.surf = pygame.image.load('Spike90.bmp').convert()
        elif direction==2:
            self.surf = pygame.image.load('Spike180.bmp').convert()
        else:
            self.surf = pygame.image.load('Spike270.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass
    
class Exit():
    rects = []
    def __init__(self, posx, posy):
        self.surf = pygame.surface.Surface((40, 40))
        self.rect = self.surf.get_rect(midbottom=(posx, posy))
        self.surf.fill(GREY)
        Platform.rects.append(self.rect)
    def draw(self):
        screen.blit(self.surf, self.rect)
    def event(self):
        pass

The Spike and Exit classes are used in an earlier iteration of the game to include added functionality of deadly spikes the player could step on and an exit that allowed movement between levels. Since we decided to change how levels worked, and since the game was challenging enough without the added spikes, these classes were not implemented in the final game class, but were left in the final code as they were good practice in translating concepts from the C iteration of the game to its Python counterpart. These classes, like most other graphics classes, have a surface that defines how the objects look, and a rect that defines how the objects behave.

In [ ]:
class Player():
    def __init__(self):
        self.jump = False
        self.left = False
        self.right = False
        self.surf = pygame.image.load('AstroRLarge.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=(60,600))
        self.y_speed = 0
    def event(self):
        if self.jump:
            self.y_speed = -22
            self.jump = False
        if self.left and self.rect.left > 0:
            self.rect.centerx -= 5
        if self.right and self.rect.right < X:
            self.rect.centerx += 5
        self.rect.bottom += self.y_speed
        if self.on_ground():
            if self.y_speed >= 0:
                self.rect.bottom = Platform.rects[self.rect.collidelist(Platform.rects)].top + 1
                self.y_speed = 0
            else:
                self.rect.top = Platform.rects[self.rect.collidelist(Platform.rects)].bottom
                self.y_speed = 2
        else:
            self.y_speed += game.acc
    def on_ground(self):
        collision = self.rect.collidelist(Platform.rects)
        if collision > -1:
            return True
        else:
            return False
    def draw(self):
        screen.blit(self.surf, self.rect)
        

The Player class defines information about The Last Astronaut himself, or the player character. The astronaut is initialized as being stationary and in the bottom left corner of the screen, as he was in the original game. The event function allows the astronaut to jump pretty high pretty quickly, with a speed of -22, to simulate low gravity. This ability to jump is different from the original game where the astronaut could not jump and was at the whim of gravity itself. The event function also allows for left and right movement and for detection of the ground beneath the astronaut. This is important to determine if the astronaut has collided with a platform (further defined by the on_ground function) and to determine if the astronaut can jump (i.e. he can jump off of the ground, but cannot jump while already in midair.) Finally, the draw function draws the surface and rect of the astronaut to the screen using the included image files for him, which works similar to other graphics in the game.

In [ ]:
class Enemy():
    def __init__(self):
        self.surf = pygame.image.load('Alien0.bmp').convert()
        self.rect = self.surf.get_rect(midtop=(700,500))
        self.x_speed = random.randint(3, 7)
        self.y_speed = 0
    def event(self):
        self.rect.centerx += self.x_speed
        if self.rect.left <= 0 or self.rect.right >= X:
            self.x_speed *= -1
        if self.on_ground():
            self.rect.bottom = Platform.rects[self.rect.collidelist(Platform.rects)].top + 1
            self.y_speed = 0
        else:
            self.y_speed += game.acc
        self.rect.bottom += self.y_speed
        self.hit()
        if game.timer:
            game.timer = False
            self.rect.midtop = (X//2, 0)
            self.x_speed = random.randint(3, 7) * ((self.x_speed > 0) - (self.x_speed < 0))

    def on_ground(self):
        collision = self.rect.collidelist(Platform.rects)
        if collision > -1:
            return True
        else:
            return False
    def hit(self):
        if (game.player.rect.colliderect(self.rect) and
            game.player.rect.midbottom != (60, 600 )):
            game.lives -= 1
            game.player.rect.midbottom = (60, 600)
            self.rect.midbottom = (420, 140)
    def draw(self):
        screen.blit(self.surf, self.rect)



The above enemy class is similar to the player class in its draw and on_ground functions for collision detection with the ground and graphics display, but is different in how it is initialized, moves, and how it does collision with objects. The alien is initialized with a random x speed between 3 and 7, meaning it could be potentially faster or slower than the player, whose speed is set at 5. The alien also spawns in a different place, and has seemingly random movements coded in to make it difficult for the player to avoid him. The alien will turn if it hits a wall and continue moving left and right, and cannot jump. If it collides with a player, as long as the player is not at the starting position (as to prevent spawn kills) the player will lose a life and the alien will reset. Finally, the alien's speed is randomly determined on reset as well, not just when it spawns. The alien cannot be killed, only avoided, and it can move coins by touching them, as we will further see in the coin class below.

In [ ]:
class Coin():
    def __init__(self):
        self.positions = [(460,440), (280,480), (60, 600), (700, 600), (540, 320), (60,120), (140, 120)]
        self.surf = pygame.image.load('coin0.bmp').convert()
        self.rect = self.surf.get_rect(midbottom=random.choice(self.positions))
        self.count = 0
    def event(self):
        if game.player.rect.colliderect(self.rect):
            self.rect.midbottom = random.choice(self.positions)
            game.coin_count += 1
        elif game.enemy.rect.colliderect(self.rect):
            self.rect.midbottom = random.choice(self.positions)
    def draw(self):
        screen.blit(self.surf, self.rect)
        

The above coin class is initialized with a large array of possible positions on top of existing platforms that are reachable by both the alien and the astronaut. The coin event function defines how the coin will randomly change positions and add to the player's inventory if it is touched by the player, but how it will only change positions without affecting the game.coin_count if touched by the alien. The alien can move coins, but cannot take away coins already collected by the player. Like most other graphics, the draw function uses the surface and rect to put the included bitmap file into the game as a sprite.

In [ ]:
class Game():
    def __init__(self):
        self.heart_surf = pygame.image.load('SpriteAstroR0.bmp').convert()
        self.coin_surf = pygame.transform.scale(pygame.image.load('coin0.bmp').convert(), (20, 20))
        self.acc = 2
        self.timer = False
        self.level = 0
        self.state = self.startpage
    def init(self):
        self.lives = 5
        self.coin_count = 0
        self.sprites = [self]
        self.state = self.loop
        self.player = Player()
        self.sprites.append(self.player)
        self.enemy = Enemy()
        self.sprites.append(self.enemy)
        self.coin = Coin()
        self.sprites.append(self.coin)
        
        for i in range(20):
            self.sprites.append(Platform(20+i*40,640))
        for i in range(13):
            self.sprites.append(Platform(20+i*40, 160))
        for i in range(4):
            self.sprites.append(Platform(500+i*40,360))
            self.sprites.append(Platform(580+i*40, 240))
        for i in range(6):
            self.sprites.append(Platform(120+i*40,520))
        for i in  range (7):
            self.sprites.append(Platform(420+i*40,480))
            self.sprites.append(Platform(20+i*40,360))
        
    
    def event(self):
        " a game state function "
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and self.player.on_ground():
                    self.player.jump = True
            elif event.type == pygame.USEREVENT + 1:
                self.timer = True
        self.player.left = False
        self.player.right = False
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.player.left = True
        if keys[pygame.K_RIGHT]:
            self.player.right = True
    def draw(self):
        for i in range(self.lives):
            screen.blit(self.heart_surf, [i*20 + 20, 20])
        for i in range(self.coin_count):
            screen.blit(self.coin_surf, [750 - i*20, 20])

    def loop(self):
        " a game state function "
        clock.tick(30)
        screen.fill(GREY)
        for s in self.sprites:
            s.event()
        for s in self.sprites:
            s.draw()
        pygame.display.flip()
        if self.lives == 0:
            self.state = self.play_again_lose
        if self.coin_count == 10:#######################level up condition
            self.state = self.play_again_win
    def play_again_win(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Win!", 1, GREEN)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click
    def play_again_lose(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Lose...", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click
    def mouse_click(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.state = self.end
                    return
                elif event.type == pygame.MOUSEBUTTONDOWN and pygame.mouse.get_pressed() == (1, 0, 0):
                    pos = pygame.mouse.get_pos()
                    for btn, action in self.buttons:
                        if btn.collidepoint(pos):
                            self.state = action
                            return
            pygame.time.wait(20)
    def end(self):
        pygame.quit()
        sys.exit()
    def startpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("The Last Astronaut", 1, BLUE)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Now", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(play, self.init)]
        pygame.display.flip()
        self.state = self.mouse_click
    def endpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("Thank you for playing", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        stop = button("Exit", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(stop, self.end)]
        pygame.display.flip()
        self.state = self.mouse_click
def button(txt, color, pos):
    button_font = pygame.font.SysFont("Segoe Print", 16)
    btn_surf = pygame.Surface((100, 40))
    btn_rect = btn_surf.get_rect(topleft=(pos))
    btn_surf.fill(GREY)
    pygame.draw.rect(btn_surf, BLACK, (0, 0, 100, 40), 1)
    txt_surf = button_font.render(txt, 1, color)
    txt_rect = txt_surf.get_rect(center=(50,20))
    btn_surf.blit(txt_surf, txt_rect)
    screen.blit(btn_surf, btn_rect)
    return btn_rect

game = Game()

while True:
    game.state()

Above is the remainder of the code, the Game class, which runs the whole game in a while loop that controls game state (the last several lines of code create a game object and start this while loop). Since this class is so large, it will be broken down and explained below a few function definitions at a time.

In [ ]:
class Game():
    def __init__(self):
        self.heart_surf = pygame.image.load('SpriteAstroR0.bmp').convert()
        self.coin_surf = pygame.transform.scale(pygame.image.load('coin0.bmp').convert(), (20, 20))
        self.acc = 2
        self.timer = False
        self.level = 0
        self.state = self.startpage
    def init(self):
        self.lives = 5
        self.coin_count = 0
        self.sprites = [self]
        self.state = self.loop
        self.player = Player()
        self.sprites.append(self.player)
        self.enemy = Enemy()
        self.sprites.append(self.enemy)
        self.coin = Coin()
        self.sprites.append(self.coin)
        
        for i in range(20):
            self.sprites.append(Platform(20+i*40,640))
        for i in range(13):
            self.sprites.append(Platform(20+i*40, 160))
        for i in range(4):
            self.sprites.append(Platform(500+i*40,360))
            self.sprites.append(Platform(580+i*40, 240))
        for i in range(6):
            self.sprites.append(Platform(120+i*40,520))
        for i in  range (7):
            self.sprites.append(Platform(420+i*40,480))
            self.sprites.append(Platform(20+i*40,360))
        
    

This code is all of the initializations to put the player, enemy, coins, and platforms in the game at the correct locations. It also initializes the level, the timer, acceleration of sprites, and provides images for the five lives and up to ten collected coins that will show status at the top of the screen. The several "for i in range" lines are initializing the location of platforms of various sizes in the game, and each large platform is made of several side-by-side brick platform objects from the Platform class we have already seen.

In [ ]:
 def event(self):
        " a game state function "
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and self.player.on_ground():
                    self.player.jump = True
            elif event.type == pygame.USEREVENT + 1:
                self.timer = True
        self.player.left = False
        self.player.right = False
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.player.left = True
        if keys[pygame.K_RIGHT]:
            self.player.right = True
    def draw(self):
        for i in range(self.lives):
            screen.blit(self.heart_surf, [i*20 + 20, 20])
        for i in range(self.coin_count):
            screen.blit(self.coin_surf, [750 - i*20, 20])


The above code gives the game state, namely whether the game is still running or if it needs to exit, as well as which, if any, keys were pressed on the keyboard and how to respond to them in game (jump, move, etc.)

In [ ]:
  def loop(self):
        " a game state function "
        clock.tick(30)
        screen.fill(GREY)
        for s in self.sprites:
            s.event()
        for s in self.sprites:
            s.draw()
        pygame.display.flip()
        if self.lives == 0:
            self.state = self.play_again_lose
        if self.coin_count == 10:#######################level up condition
            self.state = self.play_again_win
    def play_again_win(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Win!", 1, GREEN)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click
    def play_again_lose(self):
        " a game state function "
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("You Lose...", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Again", GREEN, ((X-200)//3, Y//8*5))
        stop = button("Stop", RED, ((X-200)//3*2+100, Y//8*5))
        pygame.display.flip()
        self.buttons = [(play, self.init), (stop, self.endpage)]
        self.state = self.mouse_click

This code gives the game a sense of time using the clock and checks the win condition (do you have ten coins) and the lose condition (do you have five deaths) and then, if either is true, displays the play again screen that is appropriate (either a win or lose screen). This screen will provide the user with buttons to either play again or quit, which are further explained below.

In [ ]:
def mouse_click(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.state = self.end
                    return
                elif event.type == pygame.MOUSEBUTTONDOWN and pygame.mouse.get_pressed() == (1, 0, 0):
                    pos = pygame.mouse.get_pos()
                    for btn, action in self.buttons:
                        if btn.collidepoint(pos):
                            self.state = action
                            return
            pygame.time.wait(20)
    def end(self):
        pygame.quit()
        sys.exit()

This code handles the mouse clicks on the buttons within the lose/win screens. If the player chooses to play again, the code calls self.init on the Game object and restarts without a title screen (also explained below). If the player chooses to quit, the system exits.

In [ ]:
    def startpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("The Last Astronaut", 1, BLUE)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        play = button("Play Now", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(play, self.init)]
        pygame.display.flip()
        self.state = self.mouse_click
    def endpage(self):
        " a game state function "
        screen.fill(GREY)
        font = pygame.font.SysFont("Segoe Print", 30)
        txt_surf = font.render("Thank you for playing", 1, RED)
        txt_rect = txt_surf.get_rect(center=(X//2, Y//2))
        screen.blit(txt_surf, txt_rect)
        stop = button("Exit", BLACK, ((X-100)//2, Y//8*5))
        self.buttons = [(stop, self.end)]
        pygame.display.flip()
        self.state = self.mouse_click
def button(txt, color, pos):
    button_font = pygame.font.SysFont("Segoe Print", 16)
    btn_surf = pygame.Surface((100, 40))
    btn_rect = btn_surf.get_rect(topleft=(pos))
    btn_surf.fill(GREY)
    pygame.draw.rect(btn_surf, BLACK, (0, 0, 100, 40), 1)
    txt_surf = button_font.render(txt, 1, color)
    txt_rect = txt_surf.get_rect(center=(50,20))
    btn_surf.blit(txt_surf, txt_rect)
    screen.blit(btn_surf, btn_rect)
    return btn_rect

Finally, in this last bit of code before the aforementioned game while loop, the game has access to a title page with a 'play' button and the title, The Last Astronaut, which will appear when a player starts the game for the first time, but not when they have chosen to play again from the end screen. The game also has access to the end page, which thanks the player for playing if they have chosen to quit, and provides an exit button to close the window. The button function definition helps define button functionality for all buttons that are mouse clickable an all aforementioned screens.

As promised, this is a sharable link to the videos and pictures made of the original C game: https://drive.google.com/open?id=0B8c1_ne6n5ArcEE4ajlMZ0xYLTg This is provided because the YouTube video upload of functionality has since been made private. Additionally, this is a link to our project GitHub. As of March 21st, 2019, the GitHub is also private, but settings are currently being adjusted to make it viewable: https://github.com/pandas13q/Python_Final_Project/blob/master/final_project.py Finally, the following two links are to a forum that provided us with walkthrough information about Pygame. Since we were very new to Python before beginning this class, this walkthrough was the basis for this game, but our final product is a lot different from this exact walkthrough: part 1 https://www.dreamincode.net/forums/topic/400812-walk-through-a-platform-game-made-with-pygame-part-%231/#1/ and part 2 https://www.dreamincode.net/forums/topic/400880-walk-through-a-platform-game-made-with-pygame-part-%232/#2/ Our functioning code in a folder with all image dependencies as well as screenshots of the game working will be uploaded to Canvas with this notebook. Please open the entire folder in Spyder and run the file final_project.py to play our Pygame!